# Learning Series #7A — Spatial Transcriptomics × H&E Mini-Demo

Co-visualize synthetic gene spots over a tissue image and run quick spatial stats (matplotlib-only).

## 0) Setup

In [ ]:

# !pip install pandas numpy matplotlib pillow
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from matplotlib.patches import Circle
from PIL import Image

plt.rcParams["figure.dpi"] = 140
plt.rcParams["savefig.bbox"] = "tight"

OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)
print("Saving outputs to:", OUT_DIR)


## 1) Load tissue and spots

In [ ]:

tissue_path = "example_tissue.png"
spots_path  = "example_spots.csv"
img = Image.open(tissue_path)
spots = pd.read_csv(spots_path)
assert {"x","y","CD274","GZMB","MKI67"}.issubset(spots.columns)
img.size, spots.shape


## 2) Overlay spots (single-plot figure)

In [ ]:

fig = plt.figure(figsize=(6.5,5))
plt.imshow(img)
plt.scatter(spots["x"], spots["y"], s=8, alpha=0.8, c=spots["CD274"])
plt.title("Spots over tissue — CD274 expression")
plt.axis("off")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "overlay_cd274.png"), dpi=300)
plt.savefig(os.path.join(OUT_DIR, "overlay_cd274.svg"))
plt.show()


## 3) Neighborhood summary (radius mean smoothing)

In [ ]:

R = 35
xy = spots[["x","y"]].values.astype(float)
vals = spots["MKI67"].values.astype(float)
smooth = np.zeros(len(spots))
for i in range(len(spots)):
    dx = xy[:,0]-xy[i,0]; dy = xy[:,1]-xy[i,1]
    d = np.sqrt(dx*dx+dy*dy)
    m = (d <= R)
    smooth[i] = vals[m].mean()
spots["MKI67_smooth"] = smooth
fig = plt.figure(figsize=(6.5,5))
plt.imshow(img)
plt.scatter(spots["x"], spots["y"], s=8, alpha=0.85, c=spots["MKI67_smooth"])
plt.title("Neighborhood-smoothed MKI67")
plt.axis("off")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "overlay_mki67_smooth.png"), dpi=300)
plt.savefig(os.path.join(OUT_DIR, "overlay_mki67_smooth.svg"))
plt.show()


## 4) ROI enrichment (mock tumor circle)

In [ ]:

cx, cy, rad = 400, 300, 160
dist = np.sqrt((spots["x"]-cx)**2 + (spots["y"]-cy)**2)
roi = dist <= rad
in_mean  = spots.loc[roi, "MKI67"].mean()
out_mean = spots.loc[~roi, "MKI67"].mean()
print("MKI67 mean in-ROI:", round(in_mean, 3))
print("MKI67 mean out-ROI:", round(out_mean, 3))

fig = plt.figure(figsize=(6.5,5))
plt.imshow(img)
plt.scatter(spots["x"], spots["y"], s=8, alpha=0.85, c=roi.map({True:1, False:0}))
plt.gca().add_patch(Circle((cx,cy), rad, fill=False, linewidth=2))
plt.title("ROI preview (1=in ROI)")
plt.axis("off")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "roi_preview.png"), dpi=300)
plt.savefig(os.path.join(OUT_DIR, "roi_preview.svg"))
plt.show()


## 5) Save spot table with smoothing

In [ ]:

spots.to_csv(os.path.join(OUT_DIR, "spots_with_smoothing.csv"), index=False)
os.path.exists(os.path.join(OUT_DIR, "spots_with_smoothing.csv"))
